In [2]:
from gevent import monkey; monkey.patch_all()
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..')))

from datetime import date, timedelta, datetime
import gevent
import socket
import sys
import time
import threading
import pandas as pd
import numpy as np
from scipy.signal import find_peaks, peak_prominences
from pymongo import MongoClient
import plotly.graph_objects as go

c:\users\nnnli\appdata\local\programs\python\python37-32\lib\site-packages\gevent\hub.py:154: UserWarning:

libuv only supports millisecond timer resolution; all times less will be set to 1 ms



In [3]:
from morning_server import message
from morning_server import stock_api
from morning_server import stream_readwriter
from morning.back_data import holidays
from morning.pipeline.converter import dt
from utils import time_converter
from morning.config import db

The history saving thread hit an unexpected error (This operation would block forever
	Hub: <Hub '' at 0x262a9390 default pending=0 ref=0 thread_ident=0x30f8>
	Handles:
[HandleState(handle=<cdata 'struct uv_handle_s *' 0x2524D468>, type=b'check', watcher=<loop at 0x5c44530 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x00FCB3C8>, type=b'timer', watcher=<loop at 0x5c44530 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x2524DA08>, type=b'prepare', watcher=<loop at 0x5c44530 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x2524D4C8>, type=b'check', watcher=<loop at 0x5c44530 default pending=0 ref=0>, ref=1, active=0, closing=0)]).History will not be written to the database.


In [37]:
def convert_data_readable(code, past_data):
    converted_data = []
    avg_prices = np.array([])
    for p in past_data:
        converted = dt.cybos_stock_day_tick_convert(p)
        converted['code'] = code
        avg_prices = np.append(avg_prices, np.array([converted['close_price']]))

        if len(avg_prices) == MAVG:
            converted['moving_average'] = avg_prices.mean()
            avg_prices = avg_prices[1:]
        else:
            converted['moving_average'] = avg_prices.mean()

        converted_data.append(converted)

    return converted_data

In [38]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_address = (message.SERVER_IP, message.CLIENT_SOCKET_PORT)
sock.connect(server_address)

message_reader = stream_readwriter.MessageReader(sock)
message_reader.start()

#market_code = stock_api.request_stock_code(message_reader, message.KOSPI)

DAY_WINDOW = 90
MAVG=20
# Left 45, Right 45

from_date = datetime(2018, 1, 3)
until_date = datetime(2020, 1, 7)
code = 'A005930'
past_data = stock_api.request_stock_day_data(message_reader, code, from_date - timedelta(days=int(DAY_WINDOW * 2)), until_date)

In [39]:
len(past_data)

611

In [40]:
past_data = convert_data_readable(code, past_data)
start_index = 0
for i, pd in enumerate(past_data):
    if pd['0'] == time_converter.datetime_to_intdate(from_date):
        start_index = i
        break

min_price = min([d['lowest_price'] for d in past_data])
max_price = max([d['highest_price'] for d in past_data])
min_price = min_price - min_price * 0.1
max_price = max_price + max_price * 0.1

In [83]:
annotations = []

def add_annotations(info, is_buy):
    t = info['buy_date'] if is_buy else info['sell_date']
    price = info['buy_price'] if is_buy else info['sell_price']
    if not is_buy:
        profit = "%0.2f" % ((info['sell_price'] - info['buy_price']) / info['buy_price'] * 100.0)
    text = 'B ' if is_buy else profit
    annotations.append(go.layout.Annotation(x=t,
                        y=price,
                        text=text,
                        xref='x',
                        yref='y',
                        opacity=(1.0 if not is_buy else 0.8),
                        ax=0,
                        ay=(-50 if is_buy else -75),
                        font=dict(
                                color=("black" if is_buy else 'red'),
                                size=12
                        ),
                        arrowcolor=('black' if is_buy else 'red'),
                        showarrow=True,
                        arrowhead=7))   

long_info = {'is_long': False, 'buy_date': None, 'sell_date': None, 'buy_price': 0, 'sell_price': 0}
for i, pd in enumerate(past_data[2:]):
    if not long_info['is_long']:
        if (past_data[i+1]['moving_average'] < past_data[i+1]['close_price'] and
            past_data[i]['moving_average'] > past_data[i]['close_price'] and
            pd['close_price'] > pd['moving_average']):
            long_info = {'is_long': True, 
                         'buy_date': time_converter.intdate_to_datetime(pd['0']).date(),
                         'sell_date': None,
                         'buy_price': pd['close_price'],
                         'sell_price': 0}
            add_annotations(long_info, True)
    else:
        if pd['moving_average'] > pd['close_price']:
            long_info['is_long'] = False
            long_info['sell_date'] = time_converter.intdate_to_datetime(pd['0']).date()
            long_info['sell_price'] = pd['close_price']
            add_annotations(long_info, False)


In [84]:
candle = go.Candlestick(x=[time_converter.intdate_to_datetime(d['0']).date() for d in past_data],
                open=[d['start_price'] for d in past_data],
                high=[d['highest_price'] for d in past_data],
                low=[d['lowest_price'] for d in past_data],
                close=[d['close_price'] for d in past_data],
                increasing_line_color='red', decreasing_line_color='blue')
mavg_line = go.Scatter(x=[time_converter.intdate_to_datetime(d['0']).date() for d in past_data],
                      y=[d['moving_average'] for d in past_data])

In [85]:
fig = go.Figure(data=[candle, mavg_line], layout=go.Layout(width=1920, height=768, yaxis_tickformat='d', annotations=annotations,
                yaxis=dict(range=[min_price, max_price]),))
                #updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate", args=[None])])]),)
                #frames=frame_data)
fig.update_xaxes(type="category")
fig.show()

In [61]:
print(fig)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

